In [1]:
# !pip install numpy
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

from sklearn.metrics import root_mean_squared_error

In [ ]:
# !pip install mlflow
import mlflow


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment tracking/mlruns/1', creation_time=1747168400782, experiment_id='1', last_update_time=1747168400782, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [12]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df.PULocationID + "_" + df.DOLocationID
    
    return df

In [13]:
df_train = read_dataframe('./data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2024-02.parquet')

print(df_train.shape[0], df_val.shape[0])

54373 51497


In [14]:
categorical = ['PU_DO', 'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'

pipeline = make_pipeline(
    DictVectorizer(),
    LinearRegression()
)

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
pipeline.fit(train_dicts, y_train)
y_pred = pipeline.predict(val_dicts)

print(f"root mean squared error is: {root_mean_squared_error(y_val, y_pred)}")

root mean squared error is: 5.926355548541141


In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)

In [24]:
alpha = 0.001

# mlflow ui --backend-store-uri sqlite:///mlflow.db

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2024-02.parquet")

    pipeline = make_pipeline(
        DictVectorizer(),
        Lasso(alpha)
    )
    
    mlflow.log_param("alpha", alpha)

    pipeline.fit(train_dicts, y_train)

    y_pred = pipeline.predict(val_dicts)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="./models/lin_reg.bin", artifact_path="models_pickle")

### Adding hyper-parameter tuning